In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
import os
from buildingmotif.utils import test_model_against_shapes
from buildingmotif.namespaces import BMOTIF, BRICK

## Setup BuildingMOTIF

In [2]:
file = "./test.db"
if os.path.isfile(file):
    os.remove(file) 

building_motif = BuildingMOTIF(f"sqlite:///{file}")
building_motif.setup_tables()

## Load model from file

In [3]:
medium_office_model = Model.create("http://example.org/building/")
medium_office_model.graph.parse(os.path.join('mediumOffice-validation', 'mediumOffice_brick.ttl'), format="ttl")

<Graph identifier=57c69818-176c-40c6-95b5-3649a05a3d00 (<class 'rdflib.graph.Graph'>)>

In [4]:
brick = Library.load(ontology_graph="../libraries/brick/Brick-full.ttl")
ashrae_g36 = Library.load(directory="../libraries/ashrae/guideline36/")


In [5]:
manifest = Library.load(directory="mediumOffice-validation/constraints/")

## Validate that model passes constraints

In [6]:
shape_collections = [brick.get_shape_collection(), manifest.get_shape_collection()]
validation_result, report_text, new_graph = medium_office_model.validate(shape_collections)
print(report_text)


Validation Report
Conforms: False
Results (6):
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPoint ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:class brick:Mixed_Air_Flow_Sensor ] ]
	Focus Node: <http://example.org/building/5-Zone-PVAV-2>
	Result Path: brick:hasPoint
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPoint ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:class brick:Mixed_Air_Flow_Sensor ] ]
	Focus Node: <http://example.org/building/5-Zone-PVAV-1>
	Result Path: brick:hasPoint
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: 

In [11]:
shape_collections = [brick.get_shape_collection(), ashrae_g36.get_shape_collection()]
shapes_to_test = ashrae_g36.get_shape_collection().get_shapes_of_definition_type(BMOTIF["Analytics_Application"])
conformance = test_model_against_shapes(model=medium_office_model, shape_collections=shape_collections, shapes_to_test=shapes_to_test, target_class=BRICK["AHU"])
print(conformance)

@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://example.org/building/5-Zone-PVAV> brick:hasTag <https://brickschema.org/schema/BrickTag#AHU>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1> brick:hasTag <https://brickschema.org/schema/BrickTag#AHU>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1-2spd-DX-Clg-Coil-360kBtu/hr-9.8EER> brick:hasTag <https://brickschema.org/schema/BrickTag#Coil>,
        <https://brickschema.org/schema/BrickTag#Cool>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1-2spd-DX-Clg-Coil-360kBtu/hr-9.8EER-Outlet-Air-Node_System_Node_Mass_Flow_Rate> brick:hasTag <https://brickschema.org/schema/BrickTag#Air>,
        <https://brickschema.org/schema/BrickTag#Discharge>,
        <https://brickschema.org/schema/BrickTag#Flow>,
        <https://brickschema.org/schema/BrickTag#Poi

## Fix Model

In [ ]:
# Model is missing feeds from 1 AHU to 1 VAV

# 1 VAV is missing Discharge_Air_Flow_Sensor
# apply g36 template to that VAV

## Validate and pass